# Imports and Setup

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Import your custom modules
import sys
sys.path.append('../src') # Add src directory to Python path
from data_loader import load_all_historical_data
from news_processor import load_financial_news_data, add_sentiment_score
from financial_metrics import add_daily_returns # We will specifically use daily returns here

# Configure plot styles
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 100

# Load and Prepare News Data (Sentiment Analysis)

In [19]:
print("--- Loading and Processing News Data ---")
df_news = load_financial_news_data(filepath='../data/cleaned_analyst_ratings.csv')

if df_news.empty:
    print("News data not loaded or is empty. Please check 'financial_news.csv' and data_loader.py.")
else:
    df_news_sentiment = add_sentiment_score(df_news.copy()) # Use a copy to avoid modifying original df_news
    print("\nNews data with sentiment scores:")
    print(df_news_sentiment[['date', 'stock', 'headline', 'sentiment_score']].head())

    # Aggregate sentiment by day for each stock
    # We will use the mean sentiment for simplicity, but median or sum could also be considered.
    daily_avg_sentiment = df_news_sentiment.groupby(['publication_day', 'stock'])['sentiment_score'].mean().reset_index()
    daily_avg_sentiment.rename(columns={'publication_day': 'Date', 'sentiment_score': 'daily_avg_sentiment'}, inplace=True)
    daily_avg_sentiment['Date'] = pd.to_datetime(daily_avg_sentiment['Date']) # Ensure datetime type for merging

    print("\nDaily aggregated sentiment per stock:")
    print(daily_avg_sentiment.head())

--- Loading and Processing News Data ---
Loaded financial news data from ../data/cleaned_analyst_ratings.csv

News data with sentiment scores:
                       date stock  \
0 2020-06-05 14:30:54+00:00     A   
1 2020-06-03 14:45:20+00:00     A   
2 2020-05-26 08:30:07+00:00     A   
3 2020-05-22 16:45:06+00:00     A   
4 2020-05-22 15:38:59+00:00     A   

                                            headline  sentiment_score  
0            Stocks That Hit 52-Week Highs On Friday             0.16  
1         Stocks That Hit 52-Week Highs On Wednesday             0.16  
2                      71 Biggest Movers From Friday             0.00  
3       46 Stocks Moving In Friday's Mid-Day Session             0.00  
4  B of A Securities Maintains Neutral on Agilent...             0.00  

Daily aggregated sentiment per stock:
                       Date stock  daily_avg_sentiment
0 2011-04-28 00:00:00+00:00   DEJ             0.000000
1 2011-04-28 00:00:00+00:00   DGP             0.00000

# Load and Prepare Stock Price Data

In [17]:
print("\n--- Loading and Processing Stock Historical Data ---")
all_stock_data = load_all_historical_data(data_dir='../data/yfinance_data')

processed_stock_returns = {}
for ticker, df_hist in all_stock_data.items():
    if df_hist.empty:
        print(f"Skipping {ticker}: Historical data is empty.")
        continue  
    # Calculate daily stock returns
    df_hist_with_returns = add_daily_returns(df_hist.copy())
    
    # Ensure 'Date' index is standardized to match news 'Date' for merging
    df_hist_with_returns.index = df_hist_with_returns.index.normalize() # Remove time component if present
    df_hist_with_returns.index.name = 'Date' # Ensure index name is 'Date' for merging

    processed_stock_returns[ticker] = df_hist_with_returns[['Close', 'Daily_Return']] # Keep relevant columns
    print(f"Processed daily returns for {ticker}.")

print("\nExample: AAPL Daily Returns Head:")
if 'AAPL' in processed_stock_returns:
    print(processed_stock_returns['AAPL'].head())


--- Loading and Processing Stock Historical Data ---

Example: AAPL Daily Returns Head:


# Merge Datasets (Data Alignment)

In [13]:
print("\n--- Merging News Sentiment and Stock Returns ---")
merged_data_for_correlation = {}

unique_tickers = daily_avg_sentiment['stock'].unique()

for ticker in unique_tickers:
    news_sentiment_for_ticker = daily_avg_sentiment[daily_avg_sentiment['stock'] == ticker]
    stock_returns_for_ticker = processed_stock_returns.get(ticker)

    if stock_returns_for_ticker is None or stock_returns_for_ticker.empty:
        print(f"No stock data found or empty for {ticker}. Skipping merge.")
        continue

    # Perform the merge. Using 'inner' join ensures only matching dates are kept.
    # This aligns the news sentiment for a given day with the stock's return on that same day.
    # Note: If news impacts *next day's* return, we'd shift the stock returns.
    df_merged = pd.merge(
        news_sentiment_for_ticker,
        stock_returns_for_ticker,
        on='Date',
        how='inner'
    )
    
    if not df_merged.empty:
        # Drop the first row of Daily_Return as it will be NaN due to pct_change()
        df_merged.dropna(subset=['Daily_Return'], inplace=True)
        if not df_merged.empty:
            merged_data_for_correlation[ticker] = df_merged
            print(f"Merged data for {ticker}. Rows: {len(df_merged)}")
        else:
            print(f"Merged data for {ticker} resulted in empty DataFrame after dropping NaNs.")
    else:
        print(f"No overlapping dates for {ticker} between news sentiment and stock returns.")

print("\nExample: Merged data for AAPL head (sentiment and returns):")
if 'AAPL' in merged_data_for_correlation:
    print(merged_data_for_correlation['AAPL'].head())


--- Merging News Sentiment and Stock Returns ---
No stock data found or empty for DEJ. Skipping merge.
No stock data found or empty for DGP. Skipping merge.
No stock data found or empty for ESR. Skipping merge.
No stock data found or empty for AIA. Skipping merge.
No stock data found or empty for GDL. Skipping merge.
No stock data found or empty for PTM. Skipping merge.
No stock data found or empty for DBP. Skipping merge.
No stock data found or empty for AGOL. Skipping merge.
No stock data found or empty for DGZ. Skipping merge.
No stock data found or empty for GLTR. Skipping merge.
No stock data found or empty for PSQ. Skipping merge.
No stock data found or empty for SCJ. Skipping merge.
No stock data found or empty for SZO. Skipping merge.
No stock data found or empty for TLH. Skipping merge.
No stock data found or empty for WITE. Skipping merge.
No stock data found or empty for CSM. Skipping merge.
No stock data found or empty for NIM. Skipping merge.
No stock data found or empty 

No stock data found or empty for BLV. Skipping merge.
No stock data found or empty for CMF. Skipping merge.
No stock data found or empty for CXA. Skipping merge.
No stock data found or empty for DLBL. Skipping merge.
No stock data found or empty for DTYL. Skipping merge.
No stock data found or empty for LTPZ. Skipping merge.
No stock data found or empty for LWC. Skipping merge.
No stock data found or empty for TIPZ. Skipping merge.
No stock data found or empty for UBT. Skipping merge.
No stock data found or empty for UST. Skipping merge.
No stock data found or empty for GSC. Skipping merge.
No stock data found or empty for LSC. Skipping merge.
No stock data found or empty for FVL. Skipping merge.
No stock data found or empty for PSP. Skipping merge.
No stock data found or empty for SRTY. Skipping merge.
No stock data found or empty for BOS. Skipping merge.
No stock data found or empty for EWV. Skipping merge.
No stock data found or empty for FFNM. Skipping merge.
No stock data found or

No stock data found or empty for PZI. Skipping merge.
No stock data found or empty for RCD. Skipping merge.
No stock data found or empty for WGA. Skipping merge.
No stock data found or empty for GTU. Skipping merge.
No stock data found or empty for UAE. Skipping merge.
No stock data found or empty for FBZ. Skipping merge.
No stock data found or empty for ADRU. Skipping merge.
No stock data found or empty for ECF. Skipping merge.
No stock data found or empty for ICLN. Skipping merge.
No stock data found or empty for PSLV. Skipping merge.
No stock data found or empty for XPL. Skipping merge.
No stock data found or empty for CHII. Skipping merge.
No stock data found or empty for DRV. Skipping merge.
No stock data found or empty for EAD. Skipping merge.
No stock data found or empty for PIM. Skipping merge.
No stock data found or empty for NXR. Skipping merge.
No stock data found or empty for VBFC. Skipping merge.
No stock data found or empty for EMXX. Skipping merge.
No stock data found or

No stock data found or empty for AFCB. Skipping merge.
No stock data found or empty for BBRC. Skipping merge.
No stock data found or empty for FRAK. Skipping merge.
No stock data found or empty for HTR. Skipping merge.
No stock data found or empty for PHO. Skipping merge.
No stock data found or empty for SDOW. Skipping merge.
No stock data found or empty for PHDG. Skipping merge.
No stock data found or empty for GRU. Skipping merge.
No stock data found or empty for IGU. Skipping merge.
No stock data found or empty for ONEF. Skipping merge.
No stock data found or empty for SJB. Skipping merge.
No stock data found or empty for UJB. Skipping merge.
No stock data found or empty for WEET. Skipping merge.
No stock data found or empty for FRI. Skipping merge.
No stock data found or empty for ACWX. Skipping merge.
No stock data found or empty for LMLP. Skipping merge.
No stock data found or empty for PBD. Skipping merge.
No stock data found or empty for IYK. Skipping merge.
No stock data found

No stock data found or empty for KNOW. Skipping merge.
No stock data found or empty for FXSG. Skipping merge.
No stock data found or empty for MNA. Skipping merge.
No stock data found or empty for VCV. Skipping merge.
No stock data found or empty for USBI. Skipping merge.
No stock data found or empty for GII. Skipping merge.
No stock data found or empty for SQQQ. Skipping merge.
No stock data found or empty for VIIX. Skipping merge.
No stock data found or empty for DBV. Skipping merge.
No stock data found or empty for EMCB. Skipping merge.
No stock data found or empty for NORW. Skipping merge.
No stock data found or empty for GURU. Skipping merge.
No stock data found or empty for VXUS. Skipping merge.
No stock data found or empty for HMG. Skipping merge.
No stock data found or empty for VB. Skipping merge.
No stock data found or empty for AADR. Skipping merge.
No stock data found or empty for ADRD. Skipping merge.
No stock data found or empty for PXSC. Skipping merge.
No stock data fou

...

No stock data found or empty for CINF. Skipping merge.
No stock data found or empty for CPA. Skipping merge.
No stock data found or empty for CTSO. Skipping merge.
No stock data found or empty for EFOI. Skipping merge.
No stock data found or empty for EWBC. Skipping merge.
No stock data found or empty for FMC. Skipping merge.
No stock data found or empty for FNV. Skipping merge.
No stock data found or empty for FSLR. Skipping merge.
No stock data found or empty for GEL. Skipping merge.
No stock data found or empty for GOLD. Skipping merge.
No stock data found or empty for INCY. Skipping merge.
No stock data found or empty for INGN. Skipping merge.
No stock data found or empty for IO. Skipping merge.
No stock data found or empty for ITW. Skipping merge.
No stock data found or empty for LBTYA. Skipping merge.
No stock data found or empty for LPSN. Skipping merge.
No stock data found or empty for MASI. Skipping merge.
No stock data found or empty for MO. Skipping merge.
No stock data foun

No stock data found or empty for AES. Skipping merge.
No stock data found or empty for ALXN. Skipping merge.
No stock data found or empty for AU. Skipping merge.
No stock data found or empty for CBAY. Skipping merge.
No stock data found or empty for CHH. Skipping merge.
No stock data found or empty for CLNY. Skipping merge.
No stock data found or empty for DUK. Skipping merge.
No stock data found or empty for EBS. Skipping merge.
No stock data found or empty for EWJ. Skipping merge.
No stock data found or empty for GDOT. Skipping merge.
No stock data found or empty for GIS. Skipping merge.
No stock data found or empty for HALO. Skipping merge.
No stock data found or empty for IFF. Skipping merge.
No stock data found or empty for IIVI. Skipping merge.
No stock data found or empty for LEN. Skipping merge.
No stock data found or empty for LH. Skipping merge.
No stock data found or empty for LSCC. Skipping merge.
No stock data found or empty for MNST. Skipping merge.
No stock data found or

No stock data found or empty for EW. Skipping merge.
No stock data found or empty for FLS. Skipping merge.
No stock data found or empty for GENE. Skipping merge.
No stock data found or empty for HA. Skipping merge.
No stock data found or empty for IBN. Skipping merge.
No stock data found or empty for IMMU. Skipping merge.
No stock data found or empty for KMX. Skipping merge.
No stock data found or empty for LINC. Skipping merge.
No stock data found or empty for LOGI. Skipping merge.
No stock data found or empty for MAS. Skipping merge.
No stock data found or empty for NBL. Skipping merge.
No stock data found or empty for NCLH. Skipping merge.
No stock data found or empty for NEM. Skipping merge.
No stock data found or empty for NGL. Skipping merge.
No stock data found or empty for NR. Skipping merge.
No stock data found or empty for PPL. Skipping merge.
No stock data found or empty for REG. Skipping merge.
No stock data found or empty for RGR. Skipping merge.
No stock data found or emp

No stock data found or empty for DGX. Skipping merge.
No stock data found or empty for ESEA. Skipping merge.
No stock data found or empty for GLNG. Skipping merge.
No stock data found or empty for H. Skipping merge.
No stock data found or empty for HEI. Skipping merge.
No stock data found or empty for KEY. Skipping merge.
No stock data found or empty for KEYS. Skipping merge.
No stock data found or empty for LOW. Skipping merge.
No stock data found or empty for MEIP. Skipping merge.
No stock data found or empty for MMM. Skipping merge.
No stock data found or empty for MT. Skipping merge.
No stock data found or empty for PENN. Skipping merge.
No stock data found or empty for PFG. Skipping merge.
No stock data found or empty for PLT. Skipping merge.
No stock data found or empty for QCOM. Skipping merge.
No stock data found or empty for RBS. Skipping merge.
No stock data found or empty for RL. Skipping merge.
No stock data found or empty for SPCB. Skipping merge.
No stock data found or em

No stock data found or empty for TTPH. Skipping merge.
No stock data found or empty for TUP. Skipping merge.
No stock data found or empty for TWTR. Skipping merge.
No stock data found or empty for WMT. Skipping merge.
No stock data found or empty for XLE. Skipping merge.
No stock data found or empty for XLF. Skipping merge.
No stock data found or empty for XLK. Skipping merge.
No stock data found or empty for AAL. Skipping merge.
No stock data found or empty for AZN. Skipping merge.
No stock data found or empty for CCL. Skipping merge.
No stock data found or empty for CPE. Skipping merge.
No stock data found or empty for CRC. Skipping merge.
No stock data found or empty for DAL. Skipping merge.
No stock data found or empty for DVP. Skipping merge.
No stock data found or empty for FB. Skipping merge.
No stock data found or empty for GILD. Skipping merge.
No stock data found or empty for JBLU. Skipping merge.
No stock data found or empty for LUV. Skipping merge.
No stock data found or em

In [ ]:
#Save correlation data
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)
for ticker, df in merged_data_for_correlation.items():
    df.to_csv(os.path.join(output_dir, f'{ticker}_merged_correlation_data.csv'))

print(f"\nSaved merged correlation data to {output_dir}")

#Save the overall correlation _df
correlation _df.to_csv(os.path.join(output_dir, 'overall_correlation_summary.csv'), index = False)
print(f"\Saved overall correlation summary to {ticker}")

# Perform Correlation Analysis

In [ ]:
print("\n--- Performing Correlation Analysis ---")
correlation_results = {}

for ticker, df_merged in merged_data_for_correlation.items():
    if not df_merged.empty:
        # Calculate Pearson correlation
        correlation = df_merged['daily_avg_sentiment'].corr(df_merged['Daily_Return'])
        correlation_results[ticker] = correlation
        print(f"Correlation for {ticker}: {correlation:.4f}")
    else:
        print(f"Skipping correlation for {ticker}: No merged data available.")

print("\n--- Summary of Correlation Results ---")
correlation_df = pd.DataFrame(list(correlation_results.items()), columns=['Ticker', 'Sentiment_vs_Daily_Return_Correlation'])
print(correlation_df.sort_values(by='Sentiment_vs_Daily_Return_Correlation', ascending=False))

# Visualize correlations using a bar chart
plt.figure(figsize=(10, 6))
sns.barplot(x='Ticker', y='Sentiment_vs_Daily_Return_Correlation', data=correlation_df, palette='viridis')
plt.title('Correlation between Daily News Sentiment and Daily Stock Returns')
plt.xlabel('Stock Ticker')
plt.ylabel('Pearson Correlation Coefficient')
plt.ylim(-1, 1) # Set y-axis limits for correlation
plt.axhline(0, color='gray', linestyle='--', linewidth=0.7)
plt.show()

# --- Optional: Visualize Scatter Plot for a specific stock ---
ticker_to_plot_corr = 'AAPL' # Choose a ticker to inspect
if ticker_to_plot_corr in merged_data_for_correlation:
    df_plot_corr = merged_data_for_correlation[ticker_to_plot_corr]
    plt.figure(figsize=(10, 7))
    sns.scatterplot(x='daily_avg_sentiment', y='Daily_Return', data=df_plot_corr, alpha=0.6)
    plt.title(f'Daily News Sentiment vs. Daily Returns for {ticker_to_plot_corr}\n(Correlation: {correlation_results.get(ticker_to_plot_corr):.4f})')
    plt.xlabel('Daily Average Sentiment Score')
    plt.ylabel('Daily Stock Return (%)')
    plt.axhline(0, color='grey', linestyle='--', linewidth=0.8)
    plt.axvline(0, color='grey', linestyle='--', linewidth=0.8)
    plt.show()
else:
    print(f"No merged data for {ticker_to_plot_corr} to plot correlation.")

# Interpretation and Next Steps

In [ ]:
print("\n--- Interpretation of Correlation Results ---")
print("A correlation coefficient close to 1 indicates a strong positive linear relationship (higher sentiment associated with higher returns).")
print("A correlation coefficient close to -1 indicates a strong negative linear relationship (higher sentiment associated with lower returns).")
print("A correlation coefficient close to 0 indicates a weak or no linear relationship.")

print("\nKey Observations from Correlation Analysis:")
# Based on the output of the correlation_df and plots, you would write your observations here.
# For example:
# - "We observe a generally positive, but weak, correlation between daily news sentiment and daily stock returns across most tickers."
# - "AAPL shows a slightly stronger positive correlation than AMZN."
# - "The correlation values are relatively low, suggesting that daily average sentiment alone might not be a strong linear predictor of daily returns."
# - "This could imply that the market reacts to news in more complex ways, or that other factors (technical indicators, macroeconomic news) are more influential."

print("\nFurther Steps and Considerations:")
print("1. Lagging Effects: Investigate if news sentiment on day t correlates with stock returns on day t+1, t+2, etc. This involves shifting the stock return column.")
print("2. News Volume: Explore the correlation between the *volume* of news articles (not just sentiment) and stock movements.")
print("3. Sentiment Aggregation: Experiment with different sentiment aggregation methods (e.g., sum of positive/negative scores, weighted average, median).")
print("4. Advanced Sentiment: Consider more nuanced sentiment analysis (e.g., financial-specific lexicons, BERT-based models) beyond TextBlob/VADER.")
print("5. Non-linear Relationships: Correlation measures linear relationships. Non-linear patterns might exist, which could be explored with more advanced statistical models or machine learning.")
print("6. Feature Engineering: Combine news sentiment with the technical indicators calculated in Quantitative_Analysis.ipynb to create a richer feature set for predictive modeling.")
print("7. Hypothesis Testing: Use statistical tests (e.g., t-tests, p-values) to determine if the observed correlations are statistically significant.")
print("8. Strategy Development: Based on the observed correlations, formulate initial hypotheses for investment strategies that leverage news sentiment.")